<a href="https://colab.research.google.com/github/EYBishal/Python-Training-Bootcamp/blob/main/16th-Dec-2025/RDD1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("RDD").getOrCreate()

In [5]:
data="""call_id,caller,receiver,city,call_type,duration_seconds,cost
C001,Amit,Rahul,Hyderabad,Local,180,2.5
C002,Neha,Arjun,Bangalore,STD,320,6.0
C003,Rahul,Pooja,Delhi,Local,60,1.0
C004,Pooja,Neha,Mumbai,ISD,900,25.0
C005,Arjun,Amit,Chennai,STD,400,7.5
C006,Sneha,Karan,Hyderabad,Local,240,3.0
C007,Karan,Sneha,Delhi,Local,120,2.0
C008,Riya,Vikas,Bangalore,STD,360,6.5
C009,Vikas,Riya,Mumbai,ISD,1100,30.0
C010,Anjali,Sanjay,Chennai,Local,90,1.5
C011,Farhan,Ayesha,Delhi,STD,420,7.0
C012,Ayesha,Farhan,Hyderabad,ISD,950,28.0
C013,Suresh,Divya,Bangalore,Local,150,2.0
C014,Divya,Suresh,Mumbai,STD,380,6.8
C015,Nikhil,Priya,Delhi,Local,200,2.8
C016,Priya,Nikhil,Chennai,STD,410,7.2
C017,Rohit,Kavya,Hyderabad,Local,170,2.3
C018,Kavya,Rohit,Bangalore,Local,140,2.1
C019,Manish,Tina,Mumbai,ISD,1000,27.0
C020,Tina,Manish,Delhi,STD,350,6.2
"""


In [6]:
with open("call_records.csv","w")as f:
  f.write(data)

In [7]:
rdd = spark.sparkContext.textFile('call_records.csv')
rdd.take(5)

['call_id,caller,receiver,city,call_type,duration_seconds,cost',
 'C001,Amit,Rahul,Hyderabad,Local,180,2.5',
 'C002,Neha,Arjun,Bangalore,STD,320,6.0',
 'C003,Rahul,Pooja,Delhi,Local,60,1.0',
 'C004,Pooja,Neha,Mumbai,ISD,900,25.0']

In [8]:
header=rdd.first()
data_rdd=rdd.filter(lambda x: x!=header)
data_rdd.take(5)

['C001,Amit,Rahul,Hyderabad,Local,180,2.5',
 'C002,Neha,Arjun,Bangalore,STD,320,6.0',
 'C003,Rahul,Pooja,Delhi,Local,60,1.0',
 'C004,Pooja,Neha,Mumbai,ISD,900,25.0',
 'C005,Arjun,Amit,Chennai,STD,400,7.5']

In [13]:
splid_rdd=data_rdd.map(lambda x:x.split(','))
splid_rdd.take(5)

[['C001', 'Amit', 'Rahul', 'Hyderabad', 'Local', '180', '2.5'],
 ['C002', 'Neha', 'Arjun', 'Bangalore', 'STD', '320', '6.0'],
 ['C003', 'Rahul', 'Pooja', 'Delhi', 'Local', '60', '1.0'],
 ['C004', 'Pooja', 'Neha', 'Mumbai', 'ISD', '900', '25.0'],
 ['C005', 'Arjun', 'Amit', 'Chennai', 'STD', '400', '7.5']]

In [27]:
call_cost_rdd=splid_rdd.map(lambda x:(x[3],float(x[6])))
call_cost_rdd.collect()

[('Hyderabad', 2.5),
 ('Bangalore', 6.0),
 ('Delhi', 1.0),
 ('Mumbai', 25.0),
 ('Chennai', 7.5),
 ('Hyderabad', 3.0),
 ('Delhi', 2.0),
 ('Bangalore', 6.5),
 ('Mumbai', 30.0),
 ('Chennai', 1.5),
 ('Delhi', 7.0),
 ('Hyderabad', 28.0),
 ('Bangalore', 2.0),
 ('Mumbai', 6.8),
 ('Delhi', 2.8),
 ('Chennai', 7.2),
 ('Hyderabad', 2.3),
 ('Bangalore', 2.1),
 ('Mumbai', 27.0),
 ('Delhi', 6.2)]

In [32]:
hihest_cost_rdd=call_cost_rdd.reduceByKey(lambda x,y:x+y)
hihest_cost_rdd.collect()
check_rdd=hihest_cost_rdd.reduce(lambda x,y:x if x[1]>y[1] else y)
check_rdd

('Mumbai', 88.8)

In [86]:
total_duration_rdd=splid_rdd.map(lambda x:(x[4],int(x[5])))
total_duration_rdd.collect()
c_rdd=total_duration_rdd.reduceByKey(lambda x,y:x+y)
c_rdd.collect()

[('Local', 1350), ('STD', 2640), ('ISD', 3950)]

In [50]:

no_calls_rdd = splid_rdd.map(lambda x: (x[3], 1)) .reduceByKey(lambda a, b: a + b)
no_calls_rdd.collect()


[('Hyderabad', 4),
 ('Delhi', 5),
 ('Mumbai', 4),
 ('Bangalore', 4),
 ('Chennai', 3)]

In [52]:
hihest_cost_rdd=call_cost_rdd.reduceByKey(lambda x,y:x+y)

total_avg_count_rdd= hihest_cost_rdd.join(no_calls_rdd).mapValues(lambda x: x[0] / x[1])
total_avg_count_rdd.collect()

[('Hyderabad', 8.95),
 ('Delhi', 3.8),
 ('Mumbai', 22.2),
 ('Bangalore', 4.15),
 ('Chennai', 5.3999999999999995)]

In [53]:
filter_rdd=call_cost_rdd.filter(lambda x:x[1]>20)
filter_rdd.collect()

[('Mumbai', 25.0), ('Mumbai', 30.0), ('Hyderabad', 28.0), ('Mumbai', 27.0)]

In [58]:

filter_isd_rdd = splid_rdd.filter(lambda x: x[4] == "ISD")
isd_calls_per_city = filter_isd_rdd.map(lambda x: (x[3], 1)).reduceByKey(lambda a, b: a + b)
isd_calls_per_city.collect()


[('Mumbai', 3), ('Hyderabad', 1)]

In [65]:
duration_highest_rdd=splid_rdd.map(lambda x:(x[0],int(x[5])))
duration_highest_rdd.collect()
check_c_rdd=duration_highest_rdd.reduce(lambda x,y:x if x[1]>y[1] else y)
check_c_rdd

('C009', 1100)

In [87]:
map_rdd_caller=splid_rdd.map(lambda x:(x[1],x[6]))
map_rdd_caller.collect()
total_rdd=map_rdd_caller.reduceByKey(lambda x,y:float(x)+float(y))
total_rdd.collect()

[('Amit', '2.5'),
 ('Pooja', '25.0'),
 ('Karan', '2.0'),
 ('Riya', '6.5'),
 ('Vikas', '30.0'),
 ('Suresh', '2.0'),
 ('Divya', '6.8'),
 ('Nikhil', '2.8'),
 ('Rohit', '2.3'),
 ('Manish', '27.0'),
 ('Tina', '6.2'),
 ('Neha', '6.0'),
 ('Rahul', '1.0'),
 ('Arjun', '7.5'),
 ('Sneha', '3.0'),
 ('Anjali', '1.5'),
 ('Farhan', '7.0'),
 ('Ayesha', '28.0'),
 ('Priya', '7.2'),
 ('Kavya', '2.1')]

In [89]:


suspicious_calls = splid_rdd.filter(lambda x: (float(x[6]) > 25) & (int(x[5]) > 900) )
suspicious_calls.collect()




[['C009', 'Vikas', 'Riya', 'Mumbai', 'ISD', '1100', '30.0'],
 ['C012', 'Ayesha', 'Farhan', 'Hyderabad', 'ISD', '950', '28.0'],
 ['C019', 'Manish', 'Tina', 'Mumbai', 'ISD', '1000', '27.0']]